In [29]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from scipy.stats import norm, sem
from scipy.interpolate import UnivariateSpline
from sklearn.calibration import calibration_curve
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats.stats import pearsonr
from numpy.random import default_rng
import random
pd.set_option('display.max_columns', 100)

In [30]:
# data preprocessing 

df = pd.read_csv('GerberGreenLarimer_APSR_2008_social_pressure.csv')
df = df[df['treatment'].isin([' Control',' Neighbors'])]

# df = pd.read_csv('GerberGreenLarimer_APSR_2008_social_pressure.csv')
df['treatment'] = np.where(df.treatment == ' Control',0,1)
df['voted'] = np.where(df.voted == 'Yes', 1, 0)
df['sex'] = np.where(df.sex == 'male',1, 0)
df['g2000'] = np.where(df.g2000 == 'yes', 1, 0)
df['g2002'] = np.where(df.g2002 == 'yes', 1, 0)
df['g2004'] = np.where(df.g2004 == 'yes', 1, 0)
df['p2000'] = np.where(df.p2000 == 'yes', 1, 0)
df['p2002'] = np.where(df.p2002 == 'yes', 1, 0)
df['p2004'] = np.where(df.p2004 == 'Yes', 1, 0)

cts_variables_names = ["yob","treatment","cluster","hh_id","hh_size","numberofnames","p2004_mean","g2004_mean"]
binary_variables_names = ["sex","g2000", "g2002", "p2000", "p2002", "p2004"]
# for column in binary_variables_names:
#     if column == 'sex':
#         df[column] = np.where(df[column] == ' male',1,0)
#     else:
#         df[column] = df[column].str.lower()
#         df[column] = np.where(df[column] == ' yes',1,0)
scaled_cts_covariates = StandardScaler().fit_transform(df[cts_variables_names])
binary_covariates = df[binary_variables_names]
d = pd.DataFrame(np.concatenate((scaled_cts_covariates, binary_covariates), axis=1), 
                        columns=cts_variables_names+binary_variables_names, index=df.index)
d["W"] = df["treatment"]
d["Y"] = df["voted"]


In [2]:
from metalearner import *

#### T learner

In [4]:
t_learner = Tlearner(RandomForestClassifier(),RandomForestClassifier())
t_learner.fit(X = d.drop(columns=['W','Y']), treatment = d['W'], y =  d['Y'])

In [5]:
ite, yhat_ts, yhat_cs, rmse = t_learner.get_ite(X = d.drop(columns=['W','Y']), treatment = d['W'], y =  d['Y'])

In [11]:
print(ite)

[ 0.48 -0.58 -0.56 ...  0.12 -0.31 -0.38]


In [12]:
print(yhat_ts)

[0.51 0.21 0.29 ... 0.2  0.57 0.48]


In [13]:
print(yhat_cs)

[0.03 0.79 0.85 ... 0.08 0.88 0.86]


In [9]:
def get_bootstrap_stats(boot_estimates, full_estimates, alpha=0.05):
    
    est_stat = []
    signif_level = -norm.ppf(alpha/2)
    est_boot = np.array(boot_estimates)
    stat = {}
    stat['estimator'] = 'T-learner'
    stat['estimate'] = full_estimates
    #stat['mean_boot'] = np.mean(est_boot)
    stat['SD'] = np.std(est_boot)
    stat['CI_radius'] = signif_level * stat['SD']
    stat['lower_ci'] = stat['estimate'] - stat['CI_radius']
    stat['upper_ci'] = stat['estimate'] + stat['CI_radius']
    est_stat.append(stat)

    return pd.DataFrame(est_stat)

get_bootstrap_stats(ite, 0.080)

,estimator,estimate,SD,CI_radius,lower_ci,upper_ci
0,T-learner,0.08,0.875057,1.715081,-1.635081,1.795081


#### S learner

In [21]:
s_learner = Slearner(baselearner=RandomForestClassifier())
s_learner.fit(X = d.drop(columns=['W','Y']), treatment = d['W'], y =  d['Y'])

In [22]:
ite, yhat_ts, yhat_cs, rmse = s_learner.get_ite(X = d.drop(columns=['W','Y']), treatment = d['W'], y =  d['Y'])

In [23]:
ate = np.mean(ite)

In [24]:
get_bootstrap_stats(ite, ate)

,estimator,estimate,SD,CI_radius,lower_ci,upper_ci
0,T-learner,0.037524,0.143044,0.280361,-0.242837,0.317886


### Synthetic data 

In [19]:
from resample import *

In [20]:
syn_data_class = resample_from_synthetic_data(n_sample= 1000000)
d = syn_data_class.get_data_with_same_distribution(ratio = 0.5)

In [21]:
d

,W,Y0,Y1,X,Y
0,0,1.836972,2.836972,0.836972,1.836972
1,0,1.133960,2.133960,0.133960,1.133960
2,0,1.755127,2.755127,0.755127,1.755127
3,0,1.735424,2.735424,0.735424,1.735424
4,0,1.027791,2.027791,0.027791,1.027791
...,...,...,...,...,...
999995,1,1.794883,2.794883,0.794883,2.794883
999996,1,1.203063,2.203063,0.203063,2.203063
999997,1,1.098975,2.098975,0.098975,2.098975
999998,1,1.663238,2.663238,0.663238,2.663238


In [22]:
s_learner = Slearner(baselearner=LinearRegression(), is_regressor=True)
s_learner.fit(X = np.array(d['X']), treatment = np.array(d['W']), 
              y =  np.array(d['Y']))

In [23]:
ite, yhat_ts, yhat_cs, rmse = s_learner.get_ite(X = np.array(d['X']), treatment = d['W'], y =  d['Y'])

In [24]:
ate = np.mean(ite)
get_bootstrap_stats(ite, ate)

,estimator,estimate,SD,CI_radius,lower_ci,upper_ci
0,T-learner,0.5,0.5,0.979982,-0.479982,1.479982


In [25]:
rmse

1.795962156100918e-14

In [26]:
t_learner = Tlearner(LinearRegression(),LinearRegression(), is_regressor= True)
t_learner.fit(X = np.array(d['X']).reshape(-1,1), treatment = np.array(d['W']), 
              y =  np.array(d['Y']))

In [27]:
ite, yhat_ts, yhat_cs, rmse = t_learner.get_ite(X = np.array(d['X']).reshape(-1,1), treatment = np.array(d['W']), 
              y =  np.array(d['Y']))

In [28]:
ate = np.mean(ite)
get_bootstrap_stats(ite, ate)

,estimator,estimate,SD,CI_radius,lower_ci,upper_ci
0,T-learner,1.0,1.643253e-15,3.220717e-15,1.0,1.0
